In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("PreprocessOps") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [3]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("simple_data.csv")

# ETİKET EKLEME

In [15]:
from pyspark.sql.functions import *

In [17]:
df = df.withColumn("ekonomik_durum", 
when(col("aylik_gelir")>700, 'iyi').otherwise("kotu")
                  
 )
df.toPandas().head()

sirano     isim  yas       meslek     sehir  aylik_gelir ekonomik_durum
0       1    Cemal   35         Isci    Ankara         3500            iyi
1       2    Ceyda   42        Memur   Kayseri         4200            iyi
2       3    Timur   30     Müzisyen  Istanbul         9000            iyi
3       4    Burcu   29  Pazarlamaci    Ankara         4200            iyi
4       5  Yasemin   23  Pazarlamaci     Bursa         4800            iyi

# 1. StringIndexer AŞAMASI

In [20]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler

In [21]:
meslek_indexer = StringIndexer() \
.setInputCol("meslek") \
.setOutputCol("meslek_index")

In [22]:
meslek_indexer_model = meslek_indexer.fit(df)
meslek_indexer_df = meslek_indexer_model.transform(df)

In [23]:
meslek_indexer_df.toPandas().head(15)

sirano      isim  yas       meslek     sehir  aylik_gelir ekonomik_durum  \
0        1     Cemal   35         Isci    Ankara         3500            iyi   
1        2     Ceyda   42        Memur   Kayseri         4200            iyi   
2        3     Timur   30     Müzisyen  Istanbul         9000            iyi   
3        4     Burcu   29  Pazarlamaci    Ankara         4200            iyi   
4        5   Yasemin   23  Pazarlamaci     Bursa         4800            iyi   
5        6       Ali   33        Memur    Ankara         4250            iyi   
6        7     Dilek   29  Pazarlamaci  Istanbul         7300            iyi   
7        8     Murat   31     Müzisyen  Istanbul        12000            iyi   
8        9     Ahmet   33       Doktor    Ankara        18000            iyi   
9       10  Muhittin   46       Berber  Istanbul        12000            iyi   
10      11  Hicaziye   47   Tuhafiyeci    Ankara         4800            iyi   
11      12     Harun   43      Tornacı    Ankara         4200            iyi   
12      13     Hakkı   33        Memur     Çorum         3750            iyi   
13      14   Gülizar   37       Doktor     İzmir        14250            iyi   
14      15    Şehmuz   41     Müzisyen    Ankara         8700            iyi   

    meslek_index  
0            5.0  
1            0.0  
2            1.0  
3            2.0  
4            2.0  
5            0.0  
6            2.0  
7            1.0  
8            3.0  
9            4.0  
10           7.0  
11           6.0  
12           0.0  
13           3.0  
14           1.0

In [25]:
df.groupBy(col("meslek")) \
.agg(count("*").alias("sayi")) \
.sort(desc("sayi")) \
.toPandas().head(10)

meslek  sayi
0        Memur     3
1     Müzisyen     3
2  Pazarlamaci     3
3       Doktor     2
4         Isci     1
5   Tuhafiyeci     1
6       Berber     1
7      Tornacı     1

In [26]:
sehir_indexer = StringIndexer() \
.setInputCol("sehir") \
.setOutputCol("sehir_index")

In [27]:
sehir_indexer_model = sehir_indexer.fit(meslek_indexer_df)
sehir_indexer_df = sehir_indexer_model.transform(meslek_indexer_df)

In [28]:
sehir_indexer_df.toPandas().head(15)

sirano      isim  yas       meslek     sehir  aylik_gelir ekonomik_durum  \
0        1     Cemal   35         Isci    Ankara         3500            iyi   
1        2     Ceyda   42        Memur   Kayseri         4200            iyi   
2        3     Timur   30     Müzisyen  Istanbul         9000            iyi   
3        4     Burcu   29  Pazarlamaci    Ankara         4200            iyi   
4        5   Yasemin   23  Pazarlamaci     Bursa         4800            iyi   
5        6       Ali   33        Memur    Ankara         4250            iyi   
6        7     Dilek   29  Pazarlamaci  Istanbul         7300            iyi   
7        8     Murat   31     Müzisyen  Istanbul        12000            iyi   
8        9     Ahmet   33       Doktor    Ankara        18000            iyi   
9       10  Muhittin   46       Berber  Istanbul        12000            iyi   
10      11  Hicaziye   47   Tuhafiyeci    Ankara         4800            iyi   
11      12     Harun   43      Tornacı    Ankara         4200            iyi   
12      13     Hakkı   33        Memur     Çorum         3750            iyi   
13      14   Gülizar   37       Doktor     İzmir        14250            iyi   
14      15    Şehmuz   41     Müzisyen    Ankara         8700            iyi   

    meslek_index  sehir_index  
0            5.0          0.0  
1            0.0          3.0  
2            1.0          1.0  
3            2.0          0.0  
4            2.0          2.0  
5            0.0          0.0  
6            2.0          1.0  
7            1.0          1.0  
8            3.0          0.0  
9            4.0          1.0  
10           7.0          0.0  
11           6.0          0.0  
12           0.0          4.0  
13           3.0          5.0  
14           1.0          0.0

# 2. OneHotEncoderEstimator AŞAMASI

In [30]:
encoder = OneHotEncoder() \
.setInputCols(["meslek_index","sehir_index"]) \
.setOutputCols(["meslek_encoded","sehir_encoded"])


In [31]:
encoder_model = encoder.fit(sehir_indexer_df)
encoder_df = encoder_model.transform(sehir_indexer_df)

In [32]:
encoder_df.toPandas().head()

sirano     isim  yas       meslek     sehir  aylik_gelir ekonomik_durum  \
0       1    Cemal   35         Isci    Ankara         3500            iyi   
1       2    Ceyda   42        Memur   Kayseri         4200            iyi   
2       3    Timur   30     Müzisyen  Istanbul         9000            iyi   
3       4    Burcu   29  Pazarlamaci    Ankara         4200            iyi   
4       5  Yasemin   23  Pazarlamaci     Bursa         4800            iyi   

   meslek_index  sehir_index                       meslek_encoded  \
0           5.0          0.0  (0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)   
1           0.0          3.0  (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
2           1.0          1.0  (0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
3           2.0          0.0  (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)   
4           2.0          2.0  (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)   

               sehir_encoded  
0  (1.0, 0.0, 0.0, 0.0, 0.0)  
1  (0.0, 0.0, 0.0, 1.0, 0.0)  
2  (0.0, 1.0, 0.0, 0.0, 0.0)  
3  (1.0, 0.0, 0.0, 0.0, 0.0)  
4  (0.0, 0.0, 1.0, 0.0, 0.0)

# 3. VectorAssembler AŞAMASI

In [33]:
assembler = VectorAssembler() \
.setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
.setOutputCol("vectorized_features")

In [34]:
assembler_df = assembler.transform(encoder_df)

In [35]:
# Pandas dataframe head() metodundan satır truncate'i kaldırmak için 
import pandas as pd
pd.set_option('display.max_colwidth', -1)

C:\Users\Aghayev\AppData\Local\Temp/ipykernel_11188/2458336166.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [36]:
assembler_df.select("vectorized_features").toPandas().head()

vectorized_features
0  (35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)
1  (42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)
2  (30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
3  (29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)
4  (23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)

# 4. LabelIndexer AŞAMASI

In [37]:
label_indexer = StringIndexer() \
.setInputCol("ekonomik_durum") \
.setOutputCol("label")

In [38]:
label_indexer_model = label_indexer.fit(assembler_df)
label_indexer_df = label_indexer_model.transform(assembler_df)

In [39]:
label_indexer_df.select("vectorized_features","label").toPandas().head(15)

vectorized_features  \
0   (35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
1   (42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)    
2   (30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)    
3   (29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
4   (23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)    
5   (33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
6   (29.0, 7300.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)    
7   (31.0, 12000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)   
8   (33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)   
9   (46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)   
10  (47.0, 4800.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
11  (43.0, 4200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
12  (33.0, 3750.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)    
13  (37.0, 14250.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
14  (41.0, 8700.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    

    label  
0   0.0    
1   0.0    
2   0.0    
3   0.0    
4   0.0    
5   0.0    
6   0.0    
7   0.0    
8   0.0    
9   0.0    
10  0.0    
11  0.0    
12  0.0    
13  0.0    
14  0.0

# 5. StandardScaler AŞAMASI

In [41]:
import math

In [42]:
math.sqrt(math.pow((35-33),2) + math.pow((18000-3500), 2))

14500.000137931034

In [44]:
math.sqrt(math.pow((35-33),2))

2.0

In [45]:
math.sqrt(math.pow((18000-3500), 2))

14500.0

In [46]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

In [47]:
scaler_model = scaler.fit(label_indexer_df)

In [48]:
scaler_df = scaler_model.transform(label_indexer_df)

In [49]:
scaler_df.select("features").toPandas().head(15)

features
0   (5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)
1   (6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)   
2   (4.292812263480104, 1.9859783574511787, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)   
3   (4.149718521364101, 0.9267899001438834, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)   
4   (3.29115606866808, 1.0591884573072952, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)    
5   (4.722093489828115, 0.9378231132408343, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)   
6   (4.149718521364101, 1.610849112154845, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)    
7   (4.435906005596108, 2.6479711432682382, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)   
8   (4.722093489828115, 3.9719567149023574, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0) 
9   (6.58231213733616, 2.6479711432682382, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)  
10  (6.725405879452164, 1.0591884573072952, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)                
11  (6.15303091098815, 0.9267899001438834, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)  
12  (4.722093489828115, 0.8274909822713244, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175)   
13  (5.294468458292129, 3.1444657326310326, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)                
14  (5.866843426756143, 1.9197790788694726, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)

# 6. Train-Test Ayırma AŞAMASI

In [50]:
train_df, test_df = scaler_df.randomSplit([0.8, 0.2], seed=142)

In [51]:
train_df.toPandas().head(15)

sirano      isim  yas       meslek     sehir  aylik_gelir ekonomik_durum  \
0   1       Cemal     35   Isci         Ankara    3500         iyi             
1   2       Ceyda     42   Memur        Kayseri   4200         iyi             
2   3       Timur     30   Müzisyen     Istanbul  9000         iyi             
3   5       Yasemin   23   Pazarlamaci  Bursa     4800         iyi             
4   6       Ali       33   Memur        Ankara    4250         iyi             
5   7       Dilek     29   Pazarlamaci  Istanbul  7300         iyi             
6   8       Murat     31   Müzisyen     Istanbul  12000        iyi             
7   10      Muhittin  46   Berber       Istanbul  12000        iyi             
8   12      Harun     43   Tornacı      Ankara    4200         iyi             
9   13      Hakkı     33   Memur        Çorum     3750         iyi             
10  14      Gülizar   37   Doktor       İzmir     14250        iyi             

    meslek_index  sehir_index                       meslek_encoded  \
0   5.0           0.0          (0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)   
1   0.0           3.0          (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
2   1.0           1.0          (0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
3   2.0           2.0          (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)   
4   0.0           0.0          (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
5   2.0           1.0          (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)   
6   1.0           1.0          (0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
7   4.0           1.0          (0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)   
8   6.0           0.0          (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)   
9   0.0           4.0          (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
10  3.0           5.0          (0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)   

                sehir_encoded  \
0   (1.0, 0.0, 0.0, 0.0, 0.0)   
1   (0.0, 0.0, 0.0, 1.0, 0.0)   
2   (0.0, 1.0, 0.0, 0.0, 0.0)   
3   (0.0, 0.0, 1.0, 0.0, 0.0)   
4   (1.0, 0.0, 0.0, 0.0, 0.0)   
5   (0.0, 1.0, 0.0, 0.0, 0.0)   
6   (0.0, 1.0, 0.0, 0.0, 0.0)   
7   (0.0, 1.0, 0.0, 0.0, 0.0)   
8   (1.0, 0.0, 0.0, 0.0, 0.0)   
9   (0.0, 0.0, 0.0, 0.0, 1.0)   
10  (0.0, 0.0, 0.0, 0.0, 0.0)   

                                                            vectorized_features  \
0   (35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
1   (42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)    
2   (30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)    
3   (23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)    
4   (33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
5   (29.0, 7300.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)    
6   (31.0, 12000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)   
7   (46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)   
8   (43.0, 4200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
9   (33.0, 3750.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)    
10  (37.0, 14250.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)   

    label  \
0   0.0     
1   0.0     
2   0.0     
3   0.0     
4   0.0     
5   0.0     
6   0.0     
7   0.0     
8   0.0     
9   0.0     
10  0.0     

                                                                                                                              features  
0   (5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)  
1   (6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)     
2   (4.292812263480104, 1.9859783574511787, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)     
3   (3.29115606866808, 1.0591884573072952, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.

In [52]:
test_df.toPandas().head(10)

sirano      isim  yas       meslek   sehir  aylik_gelir ekonomik_durum  \
0  4       Burcu     29   Pazarlamaci  Ankara  4200         iyi             
1  9       Ahmet     33   Doktor       Ankara  18000        iyi             
2  11      Hicaziye  47   Tuhafiyeci   Ankara  4800         iyi             
3  15      Şehmuz    41   Müzisyen     Ankara  8700         iyi             

   meslek_index  sehir_index                       meslek_encoded  \
0  2.0           0.0          (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)   
1  3.0           0.0          (0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)   
2  7.0           0.0          (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)   
3  1.0           0.0          (0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)   

               sehir_encoded  \
0  (1.0, 0.0, 0.0, 0.0, 0.0)   
1  (1.0, 0.0, 0.0, 0.0, 0.0)   
2  (1.0, 0.0, 0.0, 0.0, 0.0)   
3  (1.0, 0.0, 0.0, 0.0, 0.0)   

                                                           vectorized_features  \
0  (29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
1  (33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)   
2  (47.0, 4800.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    
3  (41.0, 8700.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)    

   label  \
0  0.0     
1  0.0     
2  0.0     
3  0.0     

                                                                                                                            features  
0  (4.149718521364101, 0.9267899001438834, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)    
1  (4.722093489828115, 3.9719567149023574, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)  
2  (6.725405879452164, 1.0591884573072952, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)                 
3  (5.866843426756143, 1.9197790788694726, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)